## Data uploading

In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Flatten , Activation , Dense , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras import regularizers
import os
import shutil
import itertools
import pathlib
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report

In [58]:
import os
import pandas as pd

train_data_path = '/Users/nikitarodionov/Downloads/archive/Training'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(train_data_path) if os.path.isdir(os.path.join(train_data_path, fold))]

for fold in folds:
    f_path = os.path.join(train_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
train_df = pd.concat([Fseries, Lseries], axis=1)
train_df

,filepaths,label
0,/Users/nikitarodionov/Downloads/archive/Traini...,pituitary
1,/Users/nikitarodionov/Downloads/archive/Traini...,pituitary
2,/Users/nikitarodionov/Downloads/archive/Traini...,pituitary
3,/Users/nikitarodionov/Downloads/archive/Traini...,pituitary
4,/Users/nikitarodionov/Downloads/archive/Traini...,pituitary
...,...,...
5707,/Users/nikitarodionov/Downloads/archive/Traini...,meningioma
5708,/Users/nikitarodionov/Downloads/archive/Traini...,meningioma
5709,/Users/nikitarodionov/Downloads/archive/Traini...,meningioma
5710,/Users/nikitarodionov/Downloads/archive/Traini...,meningioma


In [59]:
test_data_path = '/Users/nikitarodionov/Downloads/archive/Testing'

filepaths = []
labels = []

folds = [fold for fold in os.listdir(test_data_path) if os.path.isdir(os.path.join(test_data_path, fold))]

for fold in folds:
    f_path = os.path.join(test_data_path, fold)
    # Skip .DS_Store
    if not os.path.isdir(f_path):
        continue
    filelists = os.listdir(f_path)
    for file in filelists:
        filepaths.append(os.path.join(f_path, file))
        labels.append(fold)

Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='label')
test_df = pd.concat([Fseries, Lseries], axis=1)
test_df

,filepaths,label
0,/Users/nikitarodionov/Downloads/archive/Testin...,pituitary
1,/Users/nikitarodionov/Downloads/archive/Testin...,pituitary
2,/Users/nikitarodionov/Downloads/archive/Testin...,pituitary
3,/Users/nikitarodionov/Downloads/archive/Testin...,pituitary
4,/Users/nikitarodionov/Downloads/archive/Testin...,pituitary
...,...,...
1306,/Users/nikitarodionov/Downloads/archive/Testin...,meningioma
1307,/Users/nikitarodionov/Downloads/archive/Testin...,meningioma
1308,/Users/nikitarodionov/Downloads/archive/Testin...,meningioma
1309,/Users/nikitarodionov/Downloads/archive/Testin...,meningioma


## TensorFlow Image Data Generator and classes dataframes

In [60]:
img_size = (224 ,244)
batch_size = 16
valid , test = train_test_split(test_df , train_size = 0.5 , shuffle = True , random_state= 42)
tr_gen = ImageDataGenerator()
ts_gen= ImageDataGenerator()

train_gen = tr_gen.flow_from_dataframe(train_df , x_col = 'filepaths' , y_col = 'label' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode = 'rgb' , shuffle = True , batch_size =batch_size)

valid_gen = ts_gen.flow_from_dataframe(valid , x_col = 'filepaths' , y_col = 'label' , target_size = img_size , 
                                       class_mode = 'categorical',color_mode = 'rgb' , shuffle= True, batch_size = batch_size)

test_gen = ts_gen.flow_from_dataframe(test , x_col= 'filepaths' , y_col = 'label' , target_size = img_size , 
                                      class_mode = 'categorical' , color_mode= 'rgb' , shuffle = False , batch_size = batch_size)
# Classes dataframes
gen_dict = train_gen.class_indices
classes = list(gen_dict.keys())
images , labels = next(train_gen)

Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


## Model Structure

In [63]:
img_shape = (img_size[0] , img_size[1] , 3)
num_class = len(classes)

base_model = tf.keras.applications.efficientnet.EfficientNetB3(include_top = False , weights = 'imagenet' ,
                                                               input_shape = img_shape, pooling= 'max')
model = Sequential([
    base_model,
    BatchNormalization(axis= -1 , momentum= 0.99 , epsilon= 0.001),
    Dense(256, kernel_regularizer = regularizers.l2(l= 0.01) , activity_regularizer = regularizers.l1(0.005),
         bias_regularizer= regularizers.l1(0.005) , activation = 'relu'),
    Dropout(rate= 0.4 , seed = 75),
    Dense(num_class , activation = 'softmax')
])

model.compile(Adamax(learning_rate = 0.001) , loss = 'categorical_crossentropy', metrics = ['accuracy'])


## Model main info

In [64]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional  (None, 1536)              10783535  
 )                                                               
                                                                 
 batch_normalization_4 (Bat  (None, 1536)              6144      
 chNormalization)                                                
                                                                 
 dense_8 (Dense)             (None, 256)               393472    
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 11184179 (42.66 MB)
Trainable params: 11

## Iterative training

In [65]:
Epochs = 2

history = model.fit(x= train_gen , epochs = Epochs , verbose = 1 , validation_data = valid_gen ,
                   validation_steps = None , shuffle = False)

Epoch 1/2
357/357 [==============================] - 872s 2s/step - loss: 4.3635 - accuracy: 0.8676 - val_loss: 3.2808 - val_accuracy: 0.9420
Epoch 2/2
357/357 [==============================] - 856s 2s/step - loss: 2.6065 - accuracy: 0.9555 - val_loss: 2.0402 - val_accuracy: 0.9832


In [54]:
# Load the single image
image = tf.keras.utils.load_img('/Users/nikitarodionov/Downloads/archive/Testing/notumor/Te-no_0404.jpg', target_size=(img_size[0], img_size[1]))
image = tf.keras.utils.img_to_array(image)
image = tf.expand_dims(image, axis=0)
image = tf.keras.applications.efficientnet.preprocess_input(image)
predictions = model.predict(image)
predicted_class = tf.math.argmax(predictions, axis=1)
class_dict_local = {0 : 'glioma', 1 : 'meningioma', 2: 'notumor', 3 : 'pituitary'}
predicted_class_literal = class_dict_local[predicted_class.numpy()[0]]
print('The predicted class is:', predicted_class_literal)

1/1 [==============================] - 0s 66ms/step
The predicted class is: notumor


## Common Metrics

In [66]:
train_score = model.evaluate(train_gen , steps =16 , verbose = 1)
valid_score = model.evaluate(valid_gen , steps = 16 , verbose = 1)
test_score = model.evaluate(test_gen , steps = 16 , verbose = 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

16/16 [==============================] - 5s 325ms/step - loss: 2.0539 - accuracy: 0.9805
Train Loss:  2.0149800777435303
Train Accuracy:  1.0
Validation Loss:  2.0298569202423096
Validation Accuracy:  0.98828125
Test Loss:  2.0538558959960938
Test Accuracy:  0.98046875


## Metrics of classes

In [69]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)
print(classification_report(test_gen.classes, y_pred , target_names= classes ))

              precision    recall  f1-score   support

      glioma       0.98      0.95      0.96       144
  meningioma       0.96      0.98      0.97       153
     notumor       1.00      1.00      1.00       208
   pituitary       0.99      0.99      0.99       151

    accuracy                           0.98       656
   macro avg       0.98      0.98      0.98       656
weighted avg       0.98      0.98      0.98       656
